In [36]:
import pyarrow as pa
import pandas as pd
import pyarrow.plasma as plasma
import numpy as np
from datetime import datetime

df = pd.read_csv('midyear_population_age_country_code.csv')

In [37]:
client = plasma.connect("/tmp/plasma", "", 0)
df = df.iloc[1000000:2500000]
record_batch = pa.RecordBatch.from_pandas(df)

In [39]:
startTime = datetime.now()
object_id = plasma.ObjectID(np.random.bytes(20))
mock_sink = pa.MockOutputStream()
stream_writer = pa.RecordBatchStreamWriter(mock_sink, record_batch.schema)
stream_writer.write_batch(record_batch)
stream_writer.close()
data_size = mock_sink.size()
buf = client.create(object_id, data_size)
stream = pa.FixedSizeBufferWriter(buf)
stream_writer = pa.RecordBatchStreamWriter(stream, record_batch.schema)
stream_writer.write_batch(record_batch)
stream_writer.close()
client.seal(object_id)
print(object_id.binary())
print(datetime.now() - startTime)

b'nD<\r;\x1e\xc7\xc0\x83\xf5oL\xed/\x9e\xd7\x1fc\x08\xb6'
0:00:01.226766


In [25]:
print(len(df))

1000000


In [40]:
%%timeit
df[(df['country_code'] == 'CV') & (df['year'] == 1993)& (df['sex'] == 'Female')]

1 loop, best of 3: 225 ms per loop
